# Image Classification

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn import svm
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from skimage.color import rgb2grey
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import TruncatedSVD

In [3]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [4]:
def load_cifar_10_data(data_dir, negatives=False):

    meta_data_dict = unpickle(data_dir + "/batches.meta")
    cifar_label_names = meta_data_dict[b'label_names']
    cifar_label_names = np.array(cifar_label_names)

    cifar_train_data = None
    cifar_train_filenames = []
    cifar_train_labels = []

    for i in range(1, 6):
        cifar_train_data_dict = unpickle(data_dir + "/data_batch_{}".format(i))
        if i == 1:
            cifar_train_data = cifar_train_data_dict[b'data']
        else:
            cifar_train_data = np.vstack((cifar_train_data, cifar_train_data_dict[b'data']))
        cifar_train_filenames += cifar_train_data_dict[b'filenames']
        cifar_train_labels += cifar_train_data_dict[b'labels']

    cifar_train_data = cifar_train_data.reshape((len(cifar_train_data), 3, 32, 32))
    if negatives:
        cifar_train_data = cifar_train_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_train_data = np.rollaxis(cifar_train_data, 1, 4)
    cifar_train_filenames = np.array(cifar_train_filenames)
    cifar_train_labels = np.array(cifar_train_labels)

    cifar_test_data_dict = unpickle(data_dir + "/test_batch")
    cifar_test_data = cifar_test_data_dict[b'data']
    cifar_test_filenames = cifar_test_data_dict[b'filenames']
    cifar_test_labels = cifar_test_data_dict[b'labels']

    cifar_test_data = cifar_test_data.reshape((len(cifar_test_data), 3, 32, 32))
    if negatives:
        cifar_test_data = cifar_test_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        cifar_test_data = np.rollaxis(cifar_test_data, 1, 4)
    cifar_test_filenames = np.array(cifar_test_filenames)
    cifar_test_labels = np.array(cifar_test_labels)

    return cifar_train_data, cifar_train_filenames, cifar_train_labels, \
        cifar_test_data, cifar_test_filenames, cifar_test_labels, cifar_label_names

### Featurize the images as vectors that can be used for classification.

In [5]:
def create_features(img):
    color_features = img.flatten()
    grey_image = rgb2grey(img)
    hog_features = hog(grey_image, block_norm='L2-Hys', pixels_per_cell=(16, 16))
    flat_features = np.hstack(color_features)
    return flat_features

In [6]:
def create_feature_matrix(label_dataframe):
    features_list = []
    
    for img in label_dataframe:
        image_features = create_features(img)
        features_list.append(image_features)

    feature_matrix = np.array(features_list)
    return feature_matrix

In [7]:
cifar_10_dir = 'Datasets/Question-1/cifar-10-batches-py'

train_data, train_filenames, train_labels, test_data, test_filenames, test_labels, label_names = \
        load_cifar_10_data(cifar_10_dir)

In [8]:
feature_train = create_feature_matrix(train_data)
feature_test = create_feature_matrix(test_data)

# SVM Classifier

In [ ]:
classifier = svm.SVC(kernel='linear', C=2)
classifier.fit(feature_train, train_labels)
p = classifier.predict(feature_test)
accuracy_score(test_labels, p)
f1_score(test_labels, p)
confusion_matrix(test_labels, p)

### Report your observations for different values of C. 

- The C parameter tells the SVM optimization how much you want to avoid misclassifying each training example. For large values of C, the optimization will choose a smaller-margin hyperplane if that hyperplane does a better job of getting all the training points classified correctly. Conversely, a very small value of C will cause the optimizer to look for a larger-margin separating hyperplane, even if that hyperplane misclassifies more points. For very tiny values of C, you should get misclassified examples, often even if your training data is linearly separable.

In [ ]:
c = [i for i in range(1,16)]
a = []
f = []

In [ ]:
for i in c:
    classifier = svm.SVC(kernel='linear', C=i)
    classifier.fit(feature_train, train_labels)
    p = classifier.predict(feature_test)
    a.append(accuracy_score(test_labels, p))
    f.append(f1_score(test_labels, p, average='weighted'))

In [ ]:
plt.plot(c, a) 
plt.xlabel('c')
plt.ylabel('accuracy_score')
plt.title('Plot')
plt.show() 

In [ ]:
plt.plot(c, f) 
plt.xlabel('c')
plt.ylabel('f1_score')
plt.title('Plot')
plt.show() 

### Compare and contrast the classifier with the KNN classifier built in the previous assignment.

- KNN classifies data based on the distance metric whereas SVM need a proper phase of training.Generally, KNN is used as multi-class classifiers whereas standard SVM separate binary data belonging to either of one class. For a multiclass SVM, One-vs-One and One-vs-All approach is used.

### Support Vectors

In [ ]:
classifier = svm.SVC(kernel='linear', C=2)
classifier.fit(feature_train, train_labels)
p = classifier.predict(feature_test)
confusion_matrix(test_labels, p)

In [ ]:
classifier.support_vectors_